In [1]:
%load_ext autoreload
%autoreload 2
import numpy as np
from pathlib import Path
import xarray as xr
import tensorflow as tf
from shared.models import *
from shared.utilities import *
from shared.training import train_and_evaluate, split_data_on_participants, k_fold_cross_validate, compile_kwargs
from shared.normalization import *
from shared.generators import *
%env TF_FORCE_GPU_ALLOW_GROWTH=true
%env TF_GPU_ALLOCATOR=cuda_malloc_async

2023-10-09 15:13:40.375777: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-09 15:13:40.894655: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


env: TF_FORCE_GPU_ALLOW_GROWTH=true
env: TF_GPU_ALLOCATOR=cuda_malloc_async


### Set up data

In [2]:
data_path = Path("data/sat1/split_stage_data.nc")

data = xr.load_dataset(data_path)

In [2]:
data_path = Path("data/sat1/split_stage_data_unprocessed_500hz.nc")

data = xr.load_dataset(data_path)

In [8]:
data_path = Path("data/sat1/stage_data.nc")

data = xr.load_dataset(data_path)

In [3]:
train_data, val_data, test_data = split_data_on_participants(data, 60, norm_dummy)

In [52]:
model = SAT1LSTM(len(data.channels), len(data.samples), len(data.labels))
model.compile(**compile_kwargs)

In [4]:
model = SAT1GRU(len(data.channels), len(data.samples), len(data.labels))
model.compile(**compile_kwargs)

2023-10-09 15:13:49.465311: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:07:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-10-09 15:13:49.494665: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:07:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-10-09 15:13:49.494738: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:07:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-10-09 15:13:49.497199: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:07:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-10-09 15:13:49.497260: I tensorflow/compile

In [10]:
model = SAT1seq2seqGRU(len(data.channels), len(data.samples), len(data.labels))
model.compile(**compile_kwargs)

In [ ]:
train_and_evaluate(
    model,
    train_data,
    val_data,
    test_data,
    epochs=20,
    logs_path=Path("logs/"),
    generator=SequentialSAT1DataGenerator,
)

In [16]:
# Run before re-training to clear up VRAM
import gc

gc.collect()
tf.keras.backend.clear_session()
del model

In [5]:
train_and_evaluate(
    model, train_data, val_data, test_data, epochs=20, logs_path=Path("logs/")
)

Epoch 1/20


2023-10-09 15:14:00.849718: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:432] Loaded cuDNN version 8600
2023-10-09 15:14:00.990155: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:606] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
2023-10-09 15:14:01.276335: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x56155a76f3d0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-10-09 15:14:01.276374: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 3090, Compute Capability 8.6
2023-10-09 15:14:01.302761: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:255] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2023-10-09 15:14:01.501313: I ./tensorflow/compiler/jit/device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the p

758/758 [==============================] - 21s 20ms/step - loss: 1.1145 - accuracy: 0.5711 - val_loss: 0.7229 - val_accuracy: 0.7279
Epoch 2/20
758/758 [==============================] - 15s 19ms/step - loss: 0.6012 - accuracy: 0.7848 - val_loss: 0.6137 - val_accuracy: 0.7811
Epoch 3/20
758/758 [==============================] - 15s 19ms/step - loss: 0.4416 - accuracy: 0.8371 - val_loss: 0.5314 - val_accuracy: 0.8060
Epoch 4/20
758/758 [==============================] - 15s 19ms/step - loss: 0.3690 - accuracy: 0.8672 - val_loss: 0.5514 - val_accuracy: 0.8030
Epoch 5/20
758/758 [==============================] - 15s 19ms/step - loss: 0.3160 - accuracy: 0.8849 - val_loss: 0.5087 - val_accuracy: 0.8158
Epoch 6/20
758/758 [==============================] - 15s 19ms/step - loss: 0.2731 - accuracy: 0.9021 - val_loss: 0.5362 - val_accuracy: 0.8173
Epoch 7/20
758/758 [==============================] - 15s 19ms/step - loss: 0.2318 - accuracy: 0.9210 - val_loss: 0.5516 - val_accuracy: 0.8203
Epo

(<keras.src.callbacks.History at 0x7fa423e7be90>, None)

In [6]:
model.save("models/gru")

INFO:tensorflow:Assets written to: models/gru/assets


INFO:tensorflow:Assets written to: models/gru/assets


In [10]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 154, 30)]         0         
                                                                 
 masking (Masking)           (None, 154, 30)           0         
                                                                 
 gru (GRU)                   (None, 154, 64)           18432     
                                                                 
 gru_1 (GRU)                 (None, 154, 32)           9408      
                                                                 
 dense (Dense)               (None, 154, 5)            165       
                                                                 
Total params: 28005 (109.39 KB)
Trainable params: 28005 (109.39 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
